# Chat DPS

This program uses GPT 3.5 + Langchain to allow you to summarize and ask questions of PDF files.

Instructions:

- Run the first cell to install the requirements. These might take a minute
- Restart the Notebook Kernel (Circle Arrow Icon in Toolbar)
- Run the code. It will find PDF files in the same folder as the script



In [2]:
# Run this cell, then restart the notebook
!pip install --user -q langchain pypdf chromadb tiktoken 

In [2]:
import os
import logging
import glob
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from IPython.display import display, HTML
from ipywidgets import interact_manual, widgets
logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

readings = glob.glob("*.pdf")

display(HTML("<h1>ChatDPS</h1>"))
display(HTML("Use Chat GPT to ask questions of an assigned reading. For best results, keep your questions short and concise! ALWAYS VERIFY THE OUTPUT!"))
display(HTML("Select a reading to load:"))

def log(file, results):
    with open(file, "a") as f:
        f.write(results)

def history(file):
    with open(file, "r") as f:
        return f.read()
    
choose_reading_interact=widgets.interact_manual.options(manual_name="Q&A This reading.")
@choose_reading_interact(reading=readings)
def choose_reading(reading):
    loader = PyPDFLoader(reading)
    pages = loader.load_and_split()
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=250) 
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
    db = Chroma.from_documents(pages,embeddings)
    retriever = db.as_retriever()
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever) 
    qa_interact=widgets.interact_manual.options(manual_name="Ask!")

    chatlog = reading.lower().replace(".pdf",".log")

    qabox = widgets.Textarea()
    display(HTML("Ask Your Questions:"))

    @qa_interact(question=qabox)
    def q_and_a(question):
        answer = qa.run(question)
        results = f"<p><b>Question:</b><br>{question}\n"
        results += f"<p><b>Answer:</b><br><code>{answer}</code>\n<hr>\n"
        log(chatlog, results)
        display(HTML(history(chatlog)))
        qabox.value = ""


interactive(children=(Dropdown(description='reading', options=('IMLS_CompLit_2017.pdf',), value='IMLS_CompLit_…